In [ ]:
import pandas as pd

items_df = pd.DataFrame({
    'Id': [302, 504, 708, 103, 343, 565],
    'Name': ['Watch', 'Camera', 'Phone', 'Shoes', 'Laptop', 'Bed'],
    'Actual_Price': [300, 400, 350, 100, 1000, 400],
    'Discount_Percentage': [10, 15, 5, 0, 2, 7]
})

print("Initial DataFrame:")
print(items_df, "\n")

items_df['Final Price'] = items_df.apply(
    lambda row: row.Actual_Price - ((row.Discount_Percentage/100)*row.Actual_Price), axis=1)

print("DataFrame after addition of new column")

print(items_df, "\n")

In [ ]:
merged_option1 = pd.merge(etf50, option1, left_on = 'Date', right_on = 'date', how = 'inner')

In [ ]:
merged_option1 = pd.merge(rate, option1, on = 'date', how = 'inner')

In [ ]:
merged_option1=[]

In [ ]:
csv_folder = r'.\data\50etf_option_data_csv_cleaned'
output_folder = r'.\data\unrisked_rate'
csv_list = os.listdir(csv_folder)

def change(type_name):
    if type_name == "认购":
        return "call"
    elif type_name == "认沽":
        return "put"
    else:
        return "wrong_type"

for csv_name in csv_list:
    print(csv_name)
    csv_path = csv_folder +"\\"+csv_name
    output_path = output_folder + "\\" + csv_name
    data = pd.read_csv(csv_path, encoding = 'gbk', index_col= False)
    data["date"] = pd.to_datetime(data["date"])
    data = data[['date','ptmtradeday','exe_price', 'open', 'high','low','close', 'call_or_put']]
    data['call_or_put'] = data['call_or_put'].apply(change)
    merged_option1 = pd.merge(rate, data, on = 'date', how = 'inner')
    merged_option2 = pd.merge(etf50, merged_option1, left_on = 'Date', right_on = 'date', how = 'inner')
    merged_option2 = merged_option2[['call_or_put','date','ptmtradeday','exe_price','Value', 'open', 'high','low','close_y','close_x']]
    merged_option2.columns = ['call_or_put', 'date','ptmtradeday','k', 'rate', 'open', 'high','low','close','s']
    # print(merged_option2.s)
    # merged_option2['iv'] = merged_option2.apply(lambda row : iv_bs_bisection(row['s'], row['k'], row['rate'], row['ptmtradeday'], row['close'], row['call_or_put'], iv_uplimit = 1.0, iv_downlimit = 0.0001, precision = 6, year_rate = 365),  axis=1)
    merged_option2['iv'] = pd.Series
    for i in range(merged_option2.shape[1]):
        merged_option2['iv'][i] = iv_bs_bisection(np.array(merged_option2['s'][i]), np.array(merged_option2['k'][i]), np.array(merged_option2['rate'][i]), np.array(merged_option2['ptmtradeday'][i]), np.array(merged_option2['close'][i]), np.array(merged_option2['call_or_put'][i]), iv_uplimit = 1.0, iv_downlimit = 0.0001, precision = 6, year_rate = 365, max_iterations= 120)
        # merged_option2['iv'][i] = find_vol_newton(merged_option2['s'][i],merged_option2['k'][i], merged_option2['rate'][i], merged_option2['ptmtradeday'][i], merged_option2['close'][i], merged_option2['call_or_put'][i],start_sigma = 0.1, precision = 3, year_rate = 365.0, max_iterations = 100)
        print(merged_option2['s'][i],merged_option2['k'][i], merged_option2['rate'][i])
        print(merged_option2['iv'][i])
        # break
    merged_option2.to_csv(output_path)
    break
    # data processing
    # data.to_csv(output_path)


In [ ]:
merged_option2

In [ ]:
i = 0
merged_option2['s'][i],merged_option2['k'][i], merged_option2['rate'][i], merged_option2['ptmtradeday'][i], merged_option2['close'][i], merged_option2['call_or_put'][i]
iv_bs_bisection(merged_option2['s'][i],merged_option2['k'][i], merged_option2['rate'][i], merged_option2['ptmtradeday'][i], merged_option2['close'][i], merged_option2['call_or_put'][i], iv_uplimit = 1.0, iv_downlimit = 0.01, precision = 3, year_rate = 365)

In [ ]:
['call_or_put', 'date','ptmtradeday','exe_price','Value', 'open', 'high','low','close_y','close_x']]
['call_or_put', 'date','ptmtradeday','k',        'rate',  'open', 'high','low','close',  's']

In [ ]:
import pandas as pd
import os
import numpy as np
from scipy.stats import norm


In [ ]:
def bs_price(S,K,sigma,r,T, year_rate = 252, option_type = "call"):
    '''
    bs 期权定价 输出为期权价格
    输入格式：
    S:标的价格
    K:行权价
    sigma:波动率
    r:无风险利率
    T:年华剩余期限
    year_rate:年化系数
    '''

    T = T/year_rate
    d1 = (np.log(S/K) + (r + pow(sigma,2)/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    if option_type == 'call':
        # return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
        return np.maximum(S - K, 0) if T==0 else S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    elif option_type == 'put':
        # return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
        return np.maximum(K-S,0) if T==0 else K*np.exp(-r*T)*norm.cdf(-1*d2) - S*norm.cdf(-1*d1)
    else:
        print("期权类型错误")
        return -1

def bs_vega(S,K,sigma,r,T ,year_rate = 252):
    '''
    计算vega
    '''
    T = T/year_rate
    d1 = (np.log(S/K) + (r + pow(sigma,2)/2)*T) / (sigma*np.sqrt(np.abs(T)) )
    return S*norm.pdf(d1)*np.sqrt(np.abs(T))
    # return S * np.sqrt(abs(T)) * np.exp(-pow(d1,2)/2) / np.sqrt(2*np.pi)

def iv_bs_bisection(S, K, r, T, price, option_type, iv_uplimit = 2.0, iv_downlimit = 0.000001, precision = 1, year_rate = 252, max_iterations = 200):
    '''二分法求iv'''
    left_iv, right_iv = iv_downlimit, iv_uplimit
    # left_price = BS_price(S,K,left_iv, r, T,  year_rate = year_rate, option_type = option_type)
    # right_price= BS_price(S,K,right_iv, r, T,  year_rate = year_rate, option_type = option_type)
    mid_iv = (left_iv + right_iv)/2
    mid_price = bs_price(S,K,mid_iv, r, T,  year_rate = year_rate, option_type = option_type)
    # print(f"first, the mid price is {mid_price}, the price is {price}")
    cnt = 0
    while abs(price - mid_price) >= 0.1**precision and cnt < max_iterations:
        if mid_price < price :
            left_iv = mid_iv
        else:
            right_iv = mid_iv
        mid_iv = (left_iv + right_iv)/2
        # if mid_iv < iv_uplimit or mid_iv > iv_downlimit:
        #     break
        # print(f"cnt = {cnt}, the left is{left_iv}, the right is{right_iv}, the mid is {mid_iv}")
        cnt += 1
        mid_price = bs_price(S,K,mid_iv, r, T,  year_rate = year_rate, option_type = option_type)
    return mid_iv


def find_vol_newton( S, K, r, T, target_value, option_type, start_sigma = 0.5, precision = 3, year_rate = 252, max_iterations = 200):
    '''迭代法求iv'''
    sigma = start_sigma
    for i in range(0, max_iterations):
        bs_price_ = bs_price(S,K,sigma,r,T, year_rate = year_rate, option_type = option_type)
        vega = bs_vega(S, K, T, r, sigma, year_rate = year_rate)*100
        diff = target_value - bs_price_  # our root
        if (abs(diff) < 0.1**precision):
            return sigma
        sigma = sigma + diff/(vega) # f(x) / f'(x)
    return sigma # value wasn't found, return best guess so far


In [ ]:
iv_bs_bisection(3.32, 3.4, 0.018,73, 0.33, 'call', iv_uplimit = 1.0, iv_downlimit = 0.0001, precision = 7, year_rate = 365.0,max_iterations = 200)

In [ ]:
find_vol_newton(3.32, 3.4, 0.018,73, 0.33, 'call', start_sigma = 0.5, precision = 7, year_rate = 365.0, max_iterations = 100)

In [ ]:
rate = pd.read_excel(r"D:\50etf期权数据与处理器\unrisked_rate.xlsx")
rate['date'] = pd.to_datetime(rate['date'])
rate['Value'] = rate['Value']/100
rate

In [ ]:
etf50 = pd.read_excel(r'D:\数据\50etf_df.xlsx')

In [ ]:
etf50 = etf50[['Date','close']]
etf50['Date'] = pd.to_datetime(etf50['Date'])
etf50

In [ ]:
merged_option1 = pd.merge(etf50, option1, left_on = 'Date', right_on = 'date', how = 'inner')

In [ ]:
merged_option1 = pd.merge(rate, option1, on = 'date', how = 'inner')

In [ ]:
merged_option1

In [ ]:
csv_folder = r'D:\50etf期权数据与处理器\50etf_option_data_csv'
output_folder = r'D:\50etf期权数据与处理器\50etf_option_data_csv_with_unrisked_rate'
csv_list = os.listdir(csv_folder)
def change(type_name):
    if type_name == "认购":
        return "call"
    elif type_name == "认沽":
        return "put"
    else:
        return "wrong_type"
for csv_name in csv_list:
    print(csv_name)
    csv_path = csv_folder +"\\"+csv_name
    output_path = output_folder + "\\" + csv_name
    data = pd.read_csv(csv_path, encoding = 'gbk', index_col= False)
    data["date"] = pd.to_datetime(data["date"])
    data = data[['date','ptmtradeday','exe_price', 'open', 'high','low','close', 'call_or_put']]
    data['call_or_put'] = data['call_or_put'].apply(change)
    merged_option1 = pd.merge(rate, data, on = 'date', how = 'inner')
    merged_option2 = pd.merge(etf50, merged_option1, left_on = 'Date', right_on = 'date', how = 'inner')
    merged_option2 = merged_option2[['call_or_put','date','ptmtradeday','exe_price','Value', 'open', 'high','low','close_y','close_x']]
    merged_option2.columns = ['call_or_put', 'date','ptmtradeday','k', 'rate', 'open', 'high','low','close','s']
    # print(merged_option2.s)
    # merged_option2['iv'] = merged_option2.apply(lambda row : iv_bs_bisection(row['s'], row['k'], row['rate'], row['ptmtradeday'], row['close'], row['call_or_put'], iv_uplimit = 1.0, iv_downlimit = 0.0001, precision = 6, year_rate = 365),  axis=1)
    merged_option2['iv'] = pd.Series
    for i in range(merged_option2.shape[1]):
        merged_option2['iv'][i] = iv_bs_bisection(np.array(merged_option2['s'][i]), np.array(merged_option2['k'][i]), np.array(merged_option2['rate'][i]), np.array(merged_option2['ptmtradeday'][i]), np.array(merged_option2['close'][i]), np.array(merged_option2['call_or_put'][i]), iv_uplimit = 1.0, iv_downlimit = 0.0001, precision = 6, year_rate = 365, max_iterations= 120)
        # merged_option2['iv'][i] = find_vol_newton(merged_option2['s'][i],merged_option2['k'][i], merged_option2['rate'][i], merged_option2['ptmtradeday'][i], merged_option2['close'][i], merged_option2['call_or_put'][i],start_sigma = 0.1, precision = 3, year_rate = 365.0, max_iterations = 100)
        print(merged_option2['s'][i],merged_option2['k'][i], merged_option2['rate'][i])
        print(merged_option2['iv'][i])
        # break
    merged_option2.to_csv(output_path)
    break
    # data processing
    # data.to_csv(output_path)


In [ ]:
merged_option2

In [ ]:
i = 0
merged_option2['s'][i],merged_option2['k'][i], merged_option2['rate'][i], merged_option2['ptmtradeday'][i], merged_option2['close'][i], merged_option2['call_or_put'][i]
iv_bs_bisection(merged_option2['s'][i],merged_option2['k'][i], merged_option2['rate'][i], merged_option2['ptmtradeday'][i], merged_option2['close'][i], merged_option2['call_or_put'][i], iv_uplimit = 1.0, iv_downlimit = 0.01, precision = 3, year_rate = 365)

In [ ]:
import pandas as pd

items_df = pd.DataFrame({
    'Id': [302, 504, 708, 103, 343, 565],
    'Name': ['Watch', 'Camera', 'Phone', 'Shoes', 'Laptop', 'Bed'],
    'Actual_Price': [300, 400, 350, 100, 1000, 400],
    'Discount_Percentage': [10, 15, 5, 0, 2, 7]
})

print("Initial DataFrame:")
print(items_df, "\n")

items_df['Final Price'] = items_df.apply(
    lambda row: row.Actual_Price - ((row.Discount_Percentage/100)*row.Actual_Price), axis=1)

print("DataFrame after addition of new column")

print(items_df, "\n")

In [ ]:
data = pd.read_csv(r'D:\50etf期权数据与处理器\50etf_option_data_csv\10000003.csv', encoding = 'gbk',index_col=False)
# data["date"] = pd.to_datetime(data["date"])

In [ ]:

np.array(merged_option2['s'])